# 0. Librerías y variables

In [1]:
import os

from dotenv import load_dotenv, dotenv_values
load_dotenv()

True

In [3]:
env_vars = dotenv_values()
for key in env_vars.keys():
    print(key)

OPENAI_API_KEY
PROXYCURL_API_KEY
TAVILY_API_KEY
LANGCHAIN_TRACING_V2
LANGCHAIN_ENDPOINT
LANGCHAIN_API_KEY
LANGCHAIN_PROJECT


# 1. OpenAI: Chat API

En este apartado se llama a la API de OpenAI directamente:

In [92]:
from openai import OpenAI

In [93]:
# Inicializar el cliente
client = OpenAI()

messages = [{"role": "user", "content": "¿Cuánto es 1+1?"}]
    
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    temperature=0.0,
)

response

ChatCompletion(id='chatcmpl-BWkfhRmicb3QmFwbJdVQAdrleJCzv', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='1+1 es igual a 2.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1747145601, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=10, prompt_tokens=17, total_tokens=27, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [94]:
response.choices[0].message.content

'1+1 es igual a 2.'

Con f-strings, se pueden crear prompts modulares, que permitan variar el valor de las variables para variar el resultado de forma sencilla:

In [95]:
# Función para parsear la respuesta
def get_completion(prompt):
    
    messages = [{"role": "user", "content": prompt}]
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0.0,
    )
    
    return response.choices[0].message.content

In [138]:
language = "Granaíno"
text = "¿De dónde vienes?"

prompt = f"Traduce el siguiente texto al {language}: {text}"

print(prompt)

Traduce el siguiente texto al Granaíno: ¿De dónde vienes?


In [137]:
get_completion(prompt)

'¿Ande vienes, chiquillo?'

# 2. LangChain: Chat API

En este apartado se va a replicar lo se ha hecho en el punto anterior, pero desde la API de LangChain. Adicionalemnte se va ahondar en 3 conceptos de LangChain:
- Modelos
- Prompts
- OutputParsers

LangChain permite llamar a diferentes modelos, con diferentes APIs, de forma agnóstica. Cmabiando ligeramente el código, puedo aprovechar una estructura ya creada para apuntar a otro modelo.

In [35]:
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage

In [101]:
# Crear los mensajes en el formato de LangChain
messages = [
    HumanMessage(content="¿Cuánto es 1+1?")
]

In [100]:
llm = ChatOllama(
    model="llama3.1",
    temperature=0.0
    )

response = llm.invoke(messages)

print(response.content)

La respuesta a la pregunta de "¿Cuánto es 1+1?" es 2.


In [166]:
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo", 
    temperature=0.0
    )

response = llm.invoke(messages)

print(response.content)

1+1 es igual a 2.


In [103]:
from langchain.chat_models import init_chat_model

# model = init_chat_model("gpt-4o-mini", model_provider="openai")
# model = init_chat_model("claude-3-5-sonnet-latest", model_provider="anthropic")
# model = init_chat_model("mistral-large-latest", model_provider="mistralai")

Las Prompt Templates (plantillas de prompt) son una herramienta de LangChain que te ayuda a crear automáticamente los mensajes que le envías al modelo de lenguaje, de una forma organizada, flexible y reutilizable.

Cuando trabajas con modelos como GPT, no sueles enviar directamente el texto del usuario al modelo. Normalmente quieres hacer algo más, como:

- Agregar instrucciones específicas (por ejemplo: "Traduce al francés...")
- Dar un contexto adicional al modelo
- Formatear el texto de forma especial
- Usar el mismo formato muchas veces con diferentes datos

Ahí es donde entran las prompt templates.

Piensa en ellas como plantillas con huecos (como los de un formulario). Tú defines una estructura fija y dejas espacios para completar con datos reales más tarde.

In [108]:
template = "Traduce el siguiente texto al {language}: {text}"

LangChain te permite definir esta plantilla y luego rellenarla automáticamente con los valores que quieras:
- `language = "francés"`
- `text = "Hola, ¿cómo estás?"`

In [142]:
from langchain_core.prompts import ChatPromptTemplate

In [139]:
# Para casos sencillos
template_string = "Traduce el siguiente texto al {language}: {text}"
prompt_template = ChatPromptTemplate.from_template(template_string)

In [157]:
# Para casos mas complejos
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "Eres un trabajador del campo, oriundo de la alpujarra de Granada."),
    ("human", "Traduce esto al {language}: {text}")
])

In [158]:
prompt_template

ChatPromptTemplate(input_variables=['language', 'text'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Eres un trabajador del campo, oriundo de la alpujarra de Granada.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language', 'text'], input_types={}, partial_variables={}, template='Traduce esto al {language}: {text}'), additional_kwargs={})])

In [159]:
prompt_template.messages[0]

SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Eres un trabajador del campo, oriundo de la alpujarra de Granada.'), additional_kwargs={})

In [160]:
prompt_template.messages[1]

HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language', 'text'], input_types={}, partial_variables={}, template='Traduce esto al {language}: {text}'), additional_kwargs={})

In [161]:
prompt_template.messages[1].prompt.input_variables

['language', 'text']

In [162]:
language = "Granaíno"
text = "¿De dónde vienes?"

translation_prompt = prompt_template.format_messages(
    language=language,
    text=text
    )

In [163]:
translation_prompt

[SystemMessage(content='Eres un trabajador del campo, oriundo de la alpujarra de Granada.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Traduce esto al Granaíno: ¿De dónde vienes?', additional_kwargs={}, response_metadata={})]

In [164]:
print(type(translation_prompt))
print(type(translation_prompt[0]))
print(type(translation_prompt[1]))

<class 'list'>
<class 'langchain_core.messages.system.SystemMessage'>
<class 'langchain_core.messages.human.HumanMessage'>


In [165]:
customer_response = llm.invoke(translation_prompt)

customer_response.content

'¿Ande vienes, majo?'

In [167]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "Eres un académico de la RAE Española."),
    ("human", "Traduce esto al {language}: {text}")
])

language = "Castellano antiguo"
text = "¿De dónde vienes?"

translation_prompt = prompt_template.format_messages(
    language=language,
    text=text
    )

customer_response = llm.invoke(translation_prompt)

customer_response.content

'¿De do vienes?'

Al implementar la llamada desde LangChain, evitamos depender de la API propia de un modelo como los de OpenAI, a utilizar la API de LangChain que es similar para todos los modelos.

Además LangChain incopora otras funciones como los OutputParsers, que mejoran la gestión de respuestas por parte de los modelos.

In [66]:
{
    "gift": False,
    "delivery_days": 5,
    "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [67]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [68]:
prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'), additional_kwargs={})]


In [69]:
prompt_template.messages[0].prompt.input_variables

['text']

In [72]:
messages = prompt_template.format_messages(text=customer_review)

llm = ChatOpenAI(
    model_name="gpt-3.5-turbo", 
    temperature=0.0
)

response = llm.invoke(messages)

print(response.content)

{
    "gift": true,
    "delivery_days": 2,
    "price_value": "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}


In [73]:
type(response.content)

str

In [74]:
# You will get an error by running this line of code 
# because'gift' is not a dictionary
# 'gift' is a string
response.content.get('gift')

AttributeError: 'str' object has no attribute 'get'

In [75]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [77]:
gift_schema = ResponseSchema(
    name="gift",
    description="Was the item purchased\
    as a gift for someone else? \
    Answer True if yes,\
    False if not or unknown."
    )

delivery_days_schema = ResponseSchema(
    name="delivery_days",
    description="How many days\
    did it take for the product\
    to arrive? If this \
    information is not found,\
    output -1."
    )

price_value_schema = ResponseSchema(
    name="price_value",
    description="Extract any\
    sentences about the value or \
    price, and output them as a \
    comma separated Python list."
    )

response_schemas = [
    gift_schema, 
    delivery_days_schema,
    price_value_schema
    ]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [78]:
format_instructions = output_parser.get_format_instructions()

In [79]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased    as a gift for someone else?     Answer True if yes,    False if not or unknown.
	"delivery_days": string  // How many days    did it take for the product    to arrive? If this     information is not found,    output -1.
	"price_value": string  // Extract any    sentences about the value or     price, and output them as a     comma separated Python list.
}
```


In [81]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(
    text=customer_review, 
    format_instructions=format_instructions
    )

In [85]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```

In [86]:
response = llm.invoke(messages)

In [87]:
print(response.content)

```json
{
	"gift": true,
	"delivery_days": 2,
	"price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}
```


In [88]:
output_dict = output_parser.parse(response.content)

In [89]:
output_dict

{'gift': True,
 'delivery_days': 2,
 'price_value': ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]}

In [90]:
type(output_dict)

dict

In [91]:
output_dict.get('delivery_days')

2

LangChain:
- Model
- Prompts
- Output Parsers